<a href="https://colab.research.google.com/github/DeepRup/DRDOReID/blob/master/PersonReID_DUKEMTMC_ReID_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import os
import random
import keras
import numpy as np 
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import *
from keras.models import *
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
!unzip DukeMTMC-reID.zip

unzip:  cannot find or open DukeMTMC-reID.zip, DukeMTMC-reID.zip.zip or DukeMTMC-reID.zip.ZIP.


In [ ]:
train_list=os.listdir("/content/DukeMTMC-reID/bounding_box_train")

l=len(train_list)
print(l)

traindf=pd.DataFrame(train_list)

train_label=[]
for n in train_list:
  arr=re.split(r'_',n)
  train_label.append(arr[0])

traindf['PersonID']=train_label
traindf.columns=['ImageName','PersonID']

print(traindf)

FileNotFoundError: ignored

In [ ]:
val_list=os.listdir("/content/DukeMTMC-reID/query")

l=len(val_list)
print(l)

valdf=pd.DataFrame(val_list)
val_label=[]

for n in val_list:
  arr=re.split(r'_',n)
  val_label.append(arr[0])

valdf['PersonID']=val_label
valdf.columns=['ImageName','PersonID']
print(valdf)

In [ ]:
base_model=tf.keras.applications.ResNet50(input_shape=(224,224,3),
                                          weights='imagenet',
                                          include_top=False)
base_model.summary()

In [ ]:
 for i, layer in enumerate(base_model.layers):
   print(i,layer.name)

In [ ]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(256,kernel_regularizer=tf.keras.regularizers.L2(0.001),
        activation='relu')(x)
x=Dense(1024,activation='relu')(x)
preds=Dense(702,activation='softmax')(x)

In [ ]:
model=Model(inputs=base_model.input,outputs=preds)
model.summary()

In [ ]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=
                                 tf.keras.applications.resnet50
                                 .preprocess_input)
val_datagen=ImageDataGenerator(preprocessing_function=
                               tf.keras.applications.resnet50
                               .preprocess_input)

In [ ]:
train_generator=train_datagen.flow_from_dataframe(traindf,
                                                 '/content/DukeMTMC-reID/bounding_box_train',
                                                  x_col='ImageName',
                                                  y_col='PersonID',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=128,
                                                  class_mode='categorical')

val_generator=val_datagen.flow_from_dataframe(valdf,
                                              '/content/DukeMTMC-reID/query',
                                              x_col='ImageName',
                                              y_col='PersonID',
                                              target_size=(224,224),
                                              color_mode='rgb',
                                              batch_size=128,
                                              class_mode='categorical')

In [ ]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history=model.fit(train_generator,epochs=5,validation_data=val_generator)